In [ ]:
df = ak.from_parquet("/eos/user/d/dvalsecc/www/ttHbb_analysis/dataset/jets_combinations_ttHbb_fullymatched_v1.parquet")

In [ ]:
(jets_fullymatched,  # initial collections of jets, already fully matched
comb_index,          # indices of jets with all the combinations for all the events
signal_comb,         # mask to get the "signal" combination  [events, True/False]
jets_combinations,   # collections of jets, many different collections for each event (from comb_index)
higgs_jets,          # higgs_jets (index 0, 1 of the combination)
tlep_jets,           # tlep jet (index 2 of the combination)
thad_jets,           # thad jets (index 3,4,5 of the combination)
whad_jets            # whad_jets (index 4 5 of the combination)
) = ak.unzip(df)

jets_combinations = ak.with_name(jets_combinations, name="Momentum4D")
higgs_jets = ak.with_name(higgs_jets, name="Momentum4D")
tlep_jets = ak.with_name(tlep_jets, name="Momentum4D")
thad_jets = ak.with_name(thad_jets, name="Momentum4D")
whad_jets = ak.with_name(whad_jets, name="Momentum4D")